In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import yfinance as yf

c:\Users\Lenovo\OneDrive\my code\AI&ML\LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_stock_data(ticker):
    data = yf.download(
        ticker,
        period="5y",
        interval="1d",
        auto_adjust=True
    )

    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [col[0] for col in data.columns]

    required_cols = ["Open", "High", "Low", "Close"]
    missing_cols = [col for col in required_cols if col not in data.columns]
    if data.empty or missing_cols:
        return None

    data = data.dropna(subset=required_cols)
    data.reset_index(inplace=True)
    return data

In [3]:
data=get_stock_data('AAPL')
data

[*********************100%***********************]  1 of 1 completed


,Date,Close,High,Low,Open,Volume
0,2020-08-18,112.409706,112.835270,110.897130,111.232718,105633600
1,2020-08-19,112.550735,113.966042,112.455899,112.818234,145538000
2,2020-08-20,115.048195,115.162490,112.575057,112.592081,126907200
3,2020-08-21,120.976921,121.460846,115.996603,116.008759,338054800
4,2020-08-24,122.423836,125.271473,120.556219,125.186351,345937600
...,...,...,...,...,...,...
1251,2025-08-12,229.649994,230.800003,227.070007,228.009995,55626200
1252,2025-08-13,233.330002,235.000000,230.429993,231.070007,69878500
1253,2025-08-14,232.779999,235.119995,230.850006,234.059998,51916300
1254,2025-08-15,231.589996,234.279999,229.339996,234.000000,56010500


In [4]:
df_prophet = data[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})


In [5]:
df_prophet.shape

(1256, 2)

In [6]:
train = df_prophet[:-200]
test = df_prophet[-200:]


In [7]:
train.shape


(1056, 2)

In [8]:
test.shape

(200, 2)

In [9]:
model = Prophet(daily_seasonality=True, yearly_seasonality=True)
model.fit(train)


23:19:26 - cmdstanpy - INFO - Chain [1] start processing
23:19:27 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
future = model.make_future_dataframe(periods=60)
forecast = model.predict(future)

In [11]:
forecast_test = forecast[['ds', 'yhat']].set_index('ds').join(test.set_index('ds'))
forecast_test.dropna(inplace=True)

In [12]:
forecast_test

,yhat,y
ds,,
2024-10-29,227.941342,232.587540
2024-10-30,228.120251,229.034073
2024-10-31,228.288141,224.863480
2024-11-01,228.729883,221.877365
2024-11-04,229.989241,220.981537
2024-11-05,230.221925,222.414871
2024-11-06,230.557213,221.688248
2024-11-07,230.877327,226.426193
2024-11-08,231.465045,226.157166


In [13]:
forecast_test = forecast_test.rename(columns={'y': 'Close'})
forecast_test.reset_index(inplace=True)
forecast_test

,ds,yhat,Close
0,2024-10-29,227.941342,232.587540
1,2024-10-30,228.120251,229.034073
2,2024-10-31,228.288141,224.863480
3,2024-11-01,228.729883,221.877365
4,2024-11-04,229.989241,220.981537
5,2024-11-05,230.221925,222.414871
6,2024-11-06,230.557213,221.688248
7,2024-11-07,230.877327,226.426193
8,2024-11-08,231.465045,226.157166
9,2024-11-11,233.107517,223.436813


In [14]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

forecast_test['Close'] = pd.to_numeric(forecast_test['Close'], errors='coerce')
forecast_test['yhat'] = pd.to_numeric(forecast_test['yhat'], errors='coerce')

forecast_test.dropna(subset=['Close', 'yhat'], inplace=True)

mae = mean_absolute_error(forecast_test['Close'], forecast_test['yhat'])
rmse = np.sqrt(mean_squared_error(forecast_test['Close'], forecast_test['yhat']))
mape = (abs((forecast_test['Close'] - forecast_test['yhat']) / forecast_test['Close'])).mean() * 100

print(f" MAE: {mae:.2f}")
print(f" RMSE: {rmse:.2f}")
print(f" MAPE: {mape:.2f}%")


 MAE: 5.76
 RMSE: 6.73
 MAPE: 2.47%


In [15]:
forecast_test

,ds,yhat,Close
0,2024-10-29,227.941342,232.587540
1,2024-10-30,228.120251,229.034073
2,2024-10-31,228.288141,224.863480
3,2024-11-01,228.729883,221.877365
4,2024-11-04,229.989241,220.981537
5,2024-11-05,230.221925,222.414871
6,2024-11-06,230.557213,221.688248
7,2024-11-07,230.877327,226.426193
8,2024-11-08,231.465045,226.157166
9,2024-11-11,233.107517,223.436813


In [16]:
print("Average Close:", forecast_test['Close'].mean())
print("Correct MAPE:", (abs((forecast_test['Close'] - forecast_test['yhat']) / forecast_test['Close'])).mean() * 100)


Average Close: 236.7088132585798
Correct MAPE: 2.4661901279588156
